# Check hint effects on homework

In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pickle as pkl
import pandas as pd
from pandas import DataFrame
from collections import defaultdict
import scipy.stats as stats
import numpy as np

## Load Data

In [15]:
hint_feature_df = pd.read_pickle("hint_feature_df.pkl")
hint_feature_df.head(3)

,"(7, 6, hint_count)","(4, 4, hint_count)","(8, 3, hint_count)","(5, 5, hint_count)","(7, 2, hint_count)","(4, 1, hint_count)","(8, 4, hint_count)","(7, 5, hint_count)","(4, 5, hint_count)","(5, 4, hint_count)",...,"(8, 2, time_spent)","(6, 6, time_spent)","(3, 1, time_spent)","(7, 3, time_spent)","(2, 12, time_spent)","(6, 5, time_spent)","(6, 4, time_spent)","(3, 5, time_spent)","(6, 3, time_spent)","(5, 1, time_spent)"
ID,,,,,,,,,,,,,,,,,,,,,
247,0,0,1,0,0,0,0,0,0,0,...,0,1160,0,3684,0,1044,0,0,6345,3444
336,0,0,0,3,0,0,0,0,0,0,...,0,960,268,0,0,0,177,0,8185,992
109,0,4,0,0,0,0,0,0,4,0,...,0,292,0,0,0,0,363,35,0,1867


In [16]:
hw_df = pd.read_pickle('hw_data.pkl')
problem_list = sorted(list(set([(c[0],c[1]) for c in hw_df.columns if len(c)==3])))
hw_df.head(1)

,"(2, 10, 1)","(2, 10, 2)","(2, 10, 3)","(2, 10, 4)","(2, 11, 1)","(2, 11, 2)","(2, 12, 1)","(2, 12, 2)","(2, 12, 3)","(2, 12, 4)",...,"(8, 3, 1)","(8, 3, 2)","(8, 3, 3)","(8, 3, 4)","(8, 4, 1)","(8, 4, 2)","(8, 4, 3)","(8, 5, 1)","(8, 5, 2)","(8, 6, 1)"
ID,,,,,,,,,,,,,,,,,,,,,
247,"[[2016-10-04 01:07:43, 1/12, 1, null]]","[[2016-10-04 01:07:43, 5/6, 1, null]]","[[2016-10-04 01:07:43, 1/12, 1, null]]","[[2016-10-04 01:07:43, 1, 1, null]]","[[2016-10-04 01:08:00, 4, 1, null]]","[[2016-10-04 01:08:00, 1/13, 1, null]]","[[2016-10-04 01:11:02, 1024, 1, null], [2016-1...","[[2016-10-04 01:11:02, 1/1024, 1, null], [2016...","[[2016-10-04 01:11:02, 10, 1, null], [2016-10-...","[[2016-10-04 01:11:02, 1/512, 0, null], [2016-...",...,"[[2016-11-17 01:10:21, 400, 0, 400/64], [2016-...","[[2016-11-17 01:19:27, (e^(-400/81))*((400/81)...","[[2016-11-17 01:19:27, (e^(-400/81))*((400/81)...","[[2016-11-17 00:22:58, 1-e^(-150), Please fin...","[[2016-11-17 02:01:33, 1/10, 1, 80.0*1/800.0],...","[[2016-11-17 02:01:33, (e^-(1/10))*((1/10)^2/(...","[[2016-11-17 02:01:33, 1-e^(-(1/10)*(1/10)), 0...","[[2016-11-17 02:22:33, (1/6.5)e^(-6.5*9), 0, e...","[[2016-11-17 03:20:07, e^(-(10/5.5))*(((10/5.5...","[[2016-11-17 02:33:13, e^(-2.1), 0, ((1/2) * e..."


In [22]:
## Get downstream problem list
downstream_problems = pkl.load(open('downstream_problems.pkl','rb'))

## Statistical Analysis

In [30]:
def ttest(control_feature, case_feature, problem_list):
    """
    perform t-test
    """
    control_mean = []
    case_mean = []
    problems = []
    p_vals = []
    for prob in problem_list:
        control = control_feature[prob]
        case = case_feature[prob]
        diff = np.mean(control)-np.mean(case)
        if control and case and diff:
            stat_values = stats.ttest_ind_from_stats(np.mean(control), np.std(control), len(control),\
                                                     np.mean(case), np.std(case), len(case))
            #stat_values = stats.ttest_ind(control, case) #this only work when control and case are of same size
            problems.append(prob)
            control_mean.append(np.mean(control))
            case_mean.append(np.mean(case))
            p_vals.append(stat_values[1])
            
            #print "{0} & {1:.3f} & {2:.3f} & {3:.3f} & {4:.3f} ".format(len(case_pro), np.mean(case_pro)/60., np.std(case_pro)/60., stat_values[1], diff/60.)
            print prob
            print "control (size/mean/std):", len(control), np.mean(control), np.std(control)
            print "case (size/mean/std):", len(case), np.mean(case), np.std(case)
            print stat_values, '\n'
    return problems, control_mean, case_mean, p_vals

## Filter functions
* statistial analysis per problem
* statistial analysis per assignment
* statistial analysis per assignment with control average

## 1. analysis per problem
* For each problem, find the number of tries(length of time) for control and case

In [31]:
def feature_per_problem(df, feature, problem_list, downstream_problems, all_control=True):
    """
    For each problem, find the number of tries(length of time) for control and case
    If [all_control] is false, only look at control students who has at least 1 hint sent.
    """
    ### for each problem find the control and case features in the format of {pro_id:[feature list]}
    control_feature = defaultdict(list)
    case_feature = defaultdict(list)

    for student_id, row in df.iterrows(): # each student
        for prob in problem_list: # each problem
            if int(prob[0]) < 4: # Skip assignment 1, 2, and 3
                continue
            h_count_key = (prob[0], prob[1], 'hint_count')
            assign_key = (prob[0],'assign')
            feature_key = (prob[0],prob[1],feature)
            prob_key = (prob[0], prob[1])
        
            if row[feature_key]:
                if all_control:
                    if row[assign_key] == '0': # control group
                        control_feature[prob].append(row[feature_key])
                else:
                    if row[assign_key] == '0' and row[h_count_key] > 0: #control group and with hints sent
                        control_feature[prob].append(row[feature_key])
                if row[assign_key] == '1': # case group
                    downstream = downstream_problems[student_id][prob[0]]
                    if downstream: # has hints in assignment prob[0]
                        if prob[1] in downstream and row[h_count_key] == 0: # prob[1] is in downstream problems
                            case_feature[prob].append(row[feature_key])

    return control_feature, case_feature

In [33]:
### Find number of tries per problem
control_feature, case_feature = feature_per_problem(hint_feature_df, "tries", problem_list, downstream_problems, all_control = False)
try_pros, try_control, try_case, try_p_vals = ttest(control_feature, case_feature, problem_list)

('4', '2')
control (size/mean/std): 10 30.0 15.3687995628
case (size/mean/std): 9 15.7777777778 9.11382073529
Ttest_indResult(statistic=2.4160805321511893, pvalue=0.027225477450614672) 

('4', '4')
control (size/mean/std): 8 9.75 3.07205143186
case (size/mean/std): 15 5.53333333333 3.3038697849
Ttest_indResult(statistic=2.9833369533560514, pvalue=0.0070864218721871019) 

('4', '5')
control (size/mean/std): 35 20.2 11.7711511756
case (size/mean/std): 8 9.25 5.58457697592
Ttest_indResult(statistic=2.5483313527828515, pvalue=0.014666315326486212) 

('4', '6')
control (size/mean/std): 1 6.0 0.0
case (size/mean/std): 33 5.15151515152 4.76172372737
Ttest_indResult(statistic=0.17554861547967993, pvalue=0.86175472838188161) 

('4', '7')
control (size/mean/std): 4 22.25 16.4829457319
case (size/mean/std): 37 9.40540540541 6.71609385935
Ttest_indResult(statistic=3.0860063732986829, pvalue=0.0037226519021434125) 

('4', '8')
control (size/mean/std): 1 16.0 0.0
case (size/mean/std): 39 18.12820512

In [34]:
### Find length of time per problem
control_feature, case_feature = feature_per_problem(hint_feature_df, "time_spent", problem_list, downstream_problems, all_control = False)
time_pros, time_control, time_case, time_p_vals = ttest(control_feature, case_feature, problem_list)

('4', '2')
control (size/mean/std): 10 4099.6 3802.47370011
case (size/mean/std): 4 4138.25 1521.82495298
Ttest_indResult(statistic=-0.019329633948673493, pvalue=0.98489582604440662) 

('4', '4')
control (size/mean/std): 8 795.125 198.800425993
case (size/mean/std): 6 706.833333333 513.741963981
Ttest_indResult(statistic=0.4482382996225785, pvalue=0.66196697053535081) 

('4', '5')
control (size/mean/std): 35 2237.77142857 1738.0339317
case (size/mean/std): 5 815.4 585.849673551
Ttest_indResult(statistic=1.7976814576881512, pvalue=0.080177131177652836) 

('4', '6')
control (size/mean/std): 1 288.0 0.0
case (size/mean/std): 18 513.944444444 500.147475165
Ttest_indResult(statistic=-0.43970665486988619, pvalue=0.66568829895562209) 

('4', '7')
control (size/mean/std): 4 2545.25 2294.89611693
case (size/mean/std): 28 1163.64285714 1251.02795487
Ttest_indResult(statistic=1.8580336267833741, pvalue=0.073002818345214604) 

('4', '8')
control (size/mean/std): 1 1603.0 0.0
case (size/mean/std): 

## 2. analysis per assignment
* For each assignment, find the number of tries(length of time) for control and case.

In [41]:
def feature_per_assignment(df, feature, problem_list, assignment_list, all_control=True):
    """
    For each assignment, find the number of tries(length of time) for control and case.
    If [all_control] is false, only look at control students who has at least 1 hint sent.
    """
    ### for each assignment find the control and case features in the format of {pro_id:[feature list]}
    control_feature = defaultdict(list)
    case_feature = defaultdict(list)

    for student_id, row in df.iterrows():
        assignment_control = defaultdict(int)
        assignment_case = defaultdict(int)
        for prob in problem_list:
            if int(prob[0]) < 4: # Skip assignment 1, 2, and 3
                continue
            h_count_key = (prob[0], prob[1], 'hint_count')
            assign_key = (prob[0],'assign')
            feature_key = (prob[0],prob[1],feature)
            pro_key = (prob[0], prob[1])
            
            if row[feature_key]:
                if row[assign_key] == '0': # control group
                    if all_control:
                        assignment_control[prob[0]] += row[feature_key]
                    else:
                        if row[h_count_key] > 0:
                            assignment_control[prob[0]] += row[feature_key]
                elif row[assign_key] == '1': # case group
                    assignment_case[prob[0]] += row[feature_key]
                    
        for assignment in assignment_list:
            if assignment_control[assignment]:
                control_feature[assignment].append(assignment_control[assignment])
            if assignment_case[assignment]:
                case_feature[assignment].append(assignment_case[assignment])

    return control_feature, case_feature

In [42]:
### Find number of tries per assignment
assignment_list = ['4', '5', '6', '7', '8']
control_feature, case_feature = feature_per_assignment(hint_feature_df, "tries", problem_list, assignment_list, all_control = False)
try_pros, try_control, try_case, try_p_vals = ttest(control_feature, case_feature, assignment_list)

4
control (size/mean/std): 53 32.7735849057 27.2947859486
case (size/mean/std): 108 101.185185185 41.7597068219
Ttest_indResult(statistic=-10.835554672167349, pvalue=7.7722369719333889e-21) 

5
control (size/mean/std): 90 83.2111111111 71.3763880105
case (size/mean/std): 106 104.886792453 71.7774759376
Ttest_indResult(statistic=-2.1122425870131338, pvalue=0.035945116499888061) 

6
control (size/mean/std): 56 53.8214285714 49.1848216798
case (size/mean/std): 111 112.585585586 48.5246233525
Ttest_indResult(statistic=-7.3548465837035666, pvalue=8.4942793433009304e-12) 

7
control (size/mean/std): 56 40.4464285714 40.46555839
case (size/mean/std): 115 138.834782609 45.8029675201
Ttest_indResult(statistic=-13.680037438697751, pvalue=3.7062525366894234e-29) 

8
control (size/mean/std): 43 23.5581395349 18.5789357164
case (size/mean/std): 101 59.0792079208 27.4459624502
Ttest_indResult(statistic=-7.7561108410599813, pvalue=1.5484934897572426e-12) 



In [43]:
### Find length of time(in seconds) per assignment
assignment_list = ['4', '5', '6', '7', '8']
control_feature, case_feature = feature_per_assignment(hint_feature_df, "time_spent", problem_list, assignment_list, all_control = False)
time_pros, time_control, time_case, time_p_vals = ttest(control_feature, case_feature, assignment_list)

4
control (size/mean/std): 53 3765.81132075 3819.05653694
case (size/mean/std): 108 9511.08333333 6055.7112882
Ttest_indResult(statistic=-6.3127214494128712, pvalue=2.6160165071685957e-09) 

5
control (size/mean/std): 90 5714.62222222 4955.3150692
case (size/mean/std): 105 6543.87619048 4637.8508666
Ttest_indResult(statistic=-1.2059662971255356, pvalue=0.22930680393019143) 

6
control (size/mean/std): 56 5547.64285714 6166.57704435
case (size/mean/std): 109 10300.2477064 8400.42630652
Ttest_indResult(statistic=-3.7447286536542044, pvalue=0.00025010249701423178) 

7
control (size/mean/std): 53 4704.45283019 5637.33822365
case (size/mean/std): 114 12677.5877193 7714.50500829
Ttest_indResult(statistic=-6.730449667660702, pvalue=2.6623703342894628e-10) 

8
control (size/mean/std): 41 3119.14634146 3334.43888987
case (size/mean/std): 89 4499.6741573 3160.48659237
Ttest_indResult(statistic=-2.2743830192528889, pvalue=0.024607137040244028) 



## 3. analysis per assignment with control average

In [67]:
def find_ave(df, feature, problem_list):
    """Find average tries for
    1. control students
    2. control students who would have got a hint
    """
    ### Average tries for all control students
    control_ave = defaultdict(float)
    
    ### Average tries for control students with a hint sent
    control_hint_ave = defaultdict(float)
    
    for pro in sorted(problem_list):
        if int(pro[0]) < 4: # skip assignment 1, 2, and 3
            continue
        hint_count_key = (pro[0], pro[1], 'hint_count')
        tries_key = (pro[0], pro[1], 'tries')
        assign_key = (pro[0], 'assign')
        prob_key = (pro[0], pro[1])
        feature_key = (pro[0], pro[1], feature)

        # all control students
        # select students who are in control and has more than 1 try.
        control_df = df[(df[assign_key] == '0') & (df[tries_key] > 0)]
        if len(control_df) != 0:
            control_ave[prob_key] = control_df[feature_key].mean()

        # control students with a hint sent
        # select students who are in control, with more than 1 try, and has hints sent
        control_hint_df = df[(df[assign_key] == '0') & (df[hint_count_key] > 0) & (df[tries_key] > 0)]
        if len(control_hint_df) != 0:
            control_hint_ave[prob_key] = control_hint_df[feature_key].mean()
            
    return control_ave, control_hint_ave

def feature_per_assignment_control_ave(df, feature, problem_list, assignment_list, downstream_problems, all_control=True):
    """
    For each assignment, find the number of tries(length of time) for downstream problems for case students.
    Also find the control average of the downstream problems for each case student.
    If [all_control] is false, only look at control students who has at least 1 hint sent.
    """
    ### for each assignment find the control and case features in the format of {pro_id:[feature list]}
    control_feature = defaultdict(list)
    case_feature = defaultdict(list)
    
    ### find control averages
    all_control_ave, control_hint_ave = find_ave(df, feature, problem_list)
    if all_control:
        control_ave = all_control_ave
    else:
        control_ave = control_hint_ave
        
    for student_id, row in df.iterrows():
        assignment_control = defaultdict(int)
        assignment_case = defaultdict(int)
        for prob in problem_list:
            if int(prob[0]) < 4: # Skip assignment 1, 2, and 3
                continue
            h_count_key = (prob[0], prob[1], 'hint_count')
            assign_key = (prob[0],'assign')
            feature_key = (prob[0],prob[1],feature)
            prob_key = (prob[0], prob[1])
                
            if row[assign_key] == '1': # case group
                downstream = downstream_problems[student_id][prob[0]]
                if downstream: # has at least one hint in assignment pro[0]                    
                    if prob[1] in downstream and \
                    row[h_count_key] == 0 and \
                    row[feature_key] and \
                    control_ave[prob_key]:
                        assignment_case[prob[0]] += row[feature_key]
                        assignment_control[prob[0]] += control_ave[prob_key]
        for assignment in assignment_list:
            if assignment_case[assignment] and assignment_control[assignment]:
                case_feature[assignment].append(assignment_case[assignment])
                control_feature[assignment].append(assignment_control[assignment])

    return control_feature, case_feature

In [68]:
### Find number of tries per assignment
assignment_list = ['4', '5', '6', '7', '8']
control_feature, case_feature = feature_per_assignment_control_ave(hint_feature_df, "tries", problem_list, assignment_list, downstream_problems, all_control = False)
try_pros, try_control, try_case, try_p_vals = ttest(control_feature, case_feature, assignment_list)

4
control (size/mean/std): 43 69.9093023256 28.4329448937
case (size/mean/std): 43 47.6744186047 23.125261462
Ttest_indResult(statistic=3.9782962499571366, pvalue=0.00014660677906989367) 

5
control (size/mean/std): 81 75.8196373166 38.8105330958
case (size/mean/std): 81 32.6913580247 25.9237447642
Ttest_indResult(statistic=8.3166034133848221, pvalue=3.723348280615239e-14) 

6
control (size/mean/std): 64 67.3930938521 34.5789562891
case (size/mean/std): 64 47.4375 37.9666280271
Ttest_indResult(statistic=3.10874899062792, pvalue=0.0023230727916378553) 

7
control (size/mean/std): 55 180.421768174 53.927291904
case (size/mean/std): 55 93.6363636364 37.4512355113
Ttest_indResult(statistic=9.8028357660332279, pvalue=1.3101589862773191e-16) 

8
control (size/mean/std): 2 23.5581395349 0.0
case (size/mean/std): 2 18.0 6.0
Ttest_indResult(statistic=1.3100660519657745, pvalue=0.32042188537743932) 



In [69]:
### Find number of tries per assignment
assignment_list = ['4', '5', '6', '7', '8']
control_feature, case_feature = feature_per_assignment_control_ave(hint_feature_df, "time_spent", problem_list, assignment_list, downstream_problems, all_control = False)
time_pros, time_control, time_case, time_p_vals = ttest(control_feature, case_feature, assignment_list)

4
control (size/mean/std): 41 5701.55088025 2457.72335143
case (size/mean/std): 41 4868.46341463 2939.880901
Ttest_indResult(statistic=1.3921006510950127, pvalue=0.16774987733796187) 

5
control (size/mean/std): 62 3918.77115541 2430.36432704
case (size/mean/std): 62 1929.29032258 1920.35198712
Ttest_indResult(statistic=5.0573869572679779, pvalue=1.5166081176193076e-06) 

6
control (size/mean/std): 58 5807.97289232 3166.44229754
case (size/mean/std): 58 4411.12068966 4643.34600699
Ttest_indResult(statistic=1.8928235608011068, pvalue=0.060917197311715805) 

7
control (size/mean/std): 53 13877.4038975 6532.43581987
case (size/mean/std): 53 8439.24528302 5868.97453325
Ttest_indResult(statistic=4.5083031295847587, pvalue=1.7196955577144108e-05) 

8
control (size/mean/std): 2 2974.06976744 0.0
case (size/mean/std): 2 882.0 222.0
Ttest_indResult(statistic=13.327177651112642, pvalue=0.0055830905995763224) 

